### [模型组网](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/02_paddle2.0_develop/04_model_cn.html)

paddle中的组网方式有两种：sequential 和 subclass 模式

In [1]:
import paddle
import warnings
warnings.filterwarnings("ignore")

#### sequential 组网
>针对顺序的线性网络结构，使用sequential可以快速组网

In [2]:
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(),
    paddle.nn.Linear(784,512),
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(512,10)
)

#### subclass 组网
>复杂的网络结构，使用Layer子类来定义模型
>
>在 init 中编写需要的层
>
>在 forward 中声明层的前向计算方式

In [3]:
class Mnist(paddle.nn.Layer):
    def __init__(self):
        super(Mnist, self).__init__()
        self.flatten = paddle.nn.Flatten()
        self.linear_1 = paddle.nn.Linear(784, 512)
        self.linear_2 = paddle.nn.Linear(512, 10)
        self.relu = paddle.nn.ReLU()
        self.dropout = paddle.nn.Dropout(0.2)
        
    def forward(self,inputs):
        y = self.flatten(inputs)
        y = self.linear_1(y)
        y = self.relu(y)
        y = self.dropout(y)
        y = self.linear_2(y)
        return y

mnist = Mnist()

#### 飞浆内置的模型

In [4]:
print("飞浆内置的模型：",paddle.vision.models.__all__)

飞浆内置的模型： ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'VGG', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'MobileNetV1', 'mobilenet_v1', 'MobileNetV2', 'mobilenet_v2', 'LeNet']


In [5]:
lenet = paddle.vision.models.LeNet()

使用 summary 查看模型的结构

In [6]:
paddle.summary(lenet,(64,1,28,28))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1      [[64, 1, 28, 28]]     [64, 6, 28, 28]          60       
    ReLU-3       [[64, 6, 28, 28]]     [64, 6, 28, 28]           0       
  MaxPool2D-1    [[64, 6, 28, 28]]     [64, 6, 14, 14]           0       
   Conv2D-2      [[64, 6, 14, 14]]     [64, 16, 10, 10]        2,416     
    ReLU-4       [[64, 16, 10, 10]]    [64, 16, 10, 10]          0       
  MaxPool2D-2    [[64, 16, 10, 10]]     [64, 16, 5, 5]           0       
   Linear-5         [[64, 400]]           [64, 120]           48,120     
   Linear-6         [[64, 120]]            [64, 84]           10,164     
   Linear-7          [[64, 84]]            [64, 10]             850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.19
Forward/backward

{'total_params': 61610, 'trainable_params': 61610}

### 训练与预测验证
>第一种：使用 paddle.Model 对模型进行封装，使用高层API进行训练和预测，比如：model.fit()、model.predict()、model.evaluate()、model.summary()等
>
>第二种：直接用基础API进行训练

In [7]:
import paddle
from paddle.vision.transforms import ToTensor

In [8]:
# 加载数据
train_dataset = paddle.vision.datasets.MNIST(mode="train",transform=ToTensor())
test_dataset = paddle.vision.datasets.MNIST(mode="test",transform=ToTensor())

In [9]:
len(train_dataset),len(test_dataset)

(60000, 10000)

#### 使用 paddle.model 训练预测

In [10]:
# 使用简单的 sequential 组网
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(1,-1),
    paddle.nn.Linear(784,512),
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(512,10)
)

# 使用 paddle.model 封装模型
model = paddle.Model(mnist)

In [11]:
# 使用高层 API 进行训练前的准备
model.prepare(
    optimizer=paddle.optimizer.Adam(parameters=model.parameters()),
    loss=paddle.nn.CrossEntropyLoss(),
    metrics=paddle.metric.Accuracy()
)

In [12]:
model.fit(
    train_dataset,
    epochs=5,
    batch_size=64,
    verbose=1
)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 938/938 [==============================] - loss: 0.0969 - acc: 0.9284 - 29ms/step          
Epoch 2/5
step 938/938 [==============================] - loss: 0.0410 - acc: 0.9688 - 27ms/step          
Epoch 3/5
step 938/938 [==============================] - loss: 0.0214 - acc: 0.9780 - 26ms/step          
Epoch 4/5
step 938/938 [==============================] - loss: 0.0052 - acc: 0.9819 - 27ms/step          
Epoch 5/5
step 938/938 [==============================] - loss: 0.0929 - acc: 0.9861 - 27ms/step          


In [13]:
eval_result = model.evaluate(test_dataset,verbose=1)

Eval begin...
step 10000/10000 [==============================] - loss: 1.7881e-06 - acc: 0.9796 - 3ms/step          - loss
Eval samples: 10000


In [14]:
test_result = model.predict(test_dataset)

Predict begin...
step 10000/10000 [==============================] - 4ms/step          
Predict samples: 10000


In [15]:
# test_result

#### 通过基础API实现模型的训练与预测

In [16]:
# 采用 subclass 组网
class Mnist(paddle.nn.Layer):
    def __init__(self):
        super(Mnist,self).__init__()
        self.flatten = paddle.nn.Flatten()
        self.linear_1 = paddle.nn.Linear(784,512)
        self.linear_2 = paddle.nn.Linear(512,10)
        self.relu = paddle.nn.ReLU()
        self.dropout = paddle.nn.Dropout(0.2)
        
    def forward(self,inputs):
        y = self.flatten(inputs)
        y = self.linear_1(y)
        y = self.relu(y)
        y = self.dropout(y)
        y = self.linear_2(y)
        return y



In [17]:
# dataset 数据集的定义不变，用 DataLoader 加载数据
train_loader = paddle.io.DataLoader(train_dataset,batch_size=64,shuffle=True)

>train_loader 总共有 938 个数据集，每个数据集中有 64 个样本
>
>采用异步加载数据，所以每调用一次返回一个数据集
>

In [18]:
len(train_loader)

938

In [19]:
mnist = Mnist()
# 开启训练模式
mnist.train()
# 设置迭代次数
epochs = 5
# 设置优化器
optim = paddle.optimizer.Adam(parameters=mnist.parameters())
# 设置损失函数
loss_fn = paddle.nn.CrossEntropyLoss()

for epoch in range(epochs):
    # 通过加载器每次随即返回64条数据
    for batch_id,data in enumerate(train_loader()):
        x_data = data[0]
        y_data = data[1]
        predicts = mnist(x_data)
        # 计算损失,等价于 prepare 中的 loss 设置
        loss = loss_fn(predicts,y_data)
        # 计算准确率，等价于 prepare 中的 metrics 设置
        acc = paddle.metric.accuracy(predicts,y_data)
        # 下面的 反向传播、更新参数、梯度清零都被封装到 Model.fit() 中
        
        # 反向传播
        loss.backward()
        if (batch_id+1)%900 ==0:
            print("epoch：{},batch_id：{},loss is：{}，acc is：{}".format(epoch,batch_id+1,loss.numpy(),acc.numpy()))
        
        # 更新参数
        optim.step()
        # 梯度清零
        optim.clear_grad()

epoch：0,batch_id：900,loss is：[0.145264]，acc is：[0.953125]
epoch：1,batch_id：900,loss is：[0.05669905]，acc is：[0.984375]
epoch：2,batch_id：900,loss is：[0.05993386]，acc is：[0.984375]
epoch：3,batch_id：900,loss is：[0.07510052]，acc is：[0.953125]
epoch：4,batch_id：900,loss is：[0.02449301]，acc is：[0.984375]


#### 用基础API验证模型

In [20]:
test_loader = paddle.io.DataLoader(test_dataset,batch_size=60,drop_last=True)
loss_fn = paddle.nn.CrossEntropyLoss()

In [21]:
# 开启验证
mnist.eval()
for batch_id,data in enumerate(test_loader()):
    x_data = data[0]
    y_data = data[1]
    predicts = mnist(x_data)
    loss = loss_fn(predicts,y_data)
    acc = paddle.metric.accuracy(predicts,y_data)
    # 打印消息
    if (batch_id+1)%30==0:
        print("batch_id：{}，loss is {}，acc is：{}".format(batch_id+1,loss.numpy(),acc.numpy()))
    



batch_id：30，loss is [0.05411525]，acc is：[0.98333335]
batch_id：60，loss is [0.11626275]，acc is：[0.98333335]
batch_id：90，loss is [0.00137621]，acc is：[1.]
batch_id：120，loss is [0.00256267]，acc is：[1.]
batch_id：150，loss is [0.01000697]，acc is：[1.]


#### 用基础API预测模型

In [22]:
test_loader = paddle.io.DataLoader(test_dataset,batch_size=64,drop_last=True)

In [23]:
mnist.eval()
for batch_id,data in enumerate(test_loader()):
    x_data = data[0]
    predicts = mnist(x_data)
print("predict finished")

predict finished
